# This project is for getting the best locality for Opening a Restaurant in desired City (In this case-Chennai, India)

let's download all the libraries that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import requests # library to handle requests
#from pandas.io.json import json_normalize
from bs4 import BeautifulSoup as soup
import matplotlib.cm as cm
import matplotlib.colors as colors
#!conda install -c conda-forge folium --yes
import folium # map rendering library
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans

### Getting the contents from the site and saving it into data variable

In [2]:
webpage = requests.get('https://finkode.com/tn/chennai.html').text
data=soup(webpage, 'lxml')

### Extracting the table from the data and saving it in pandas dataframe

In [3]:
table = data.find_all('table',  class_= 'plist' )
df = pd.read_html(str(table))
Chennai_data=pd.DataFrame(df[0])
Chennai_data['Area'] = Chennai_data[['Post Office', 'District']].apply(lambda x: ', '.join(x), axis = 1) 
Chennai_data=Chennai_data.drop(['Post Office','District'], axis=1)
Chennai_data.head(10)

,Pincode,Area
0,600029,"Aminjikarai S.O, Chennai"
1,600102,"Anna Nagar East S.O, Chennai"
2,600040,"Anna Nagar S.O (Chennai), Chennai"
3,600101,"Anna Nagar Western Extn S.O, Chennai"
4,600002,"Anna Road H.O, Chennai"
5,600106,"Arumbakkam S.O, Chennai"
6,600083,"Ashoknagar S.O (Chennai), Chennai"
7,600023,"Aynavaram S.O, Chennai"
8,600090,"Besantnagar S.O, Chennai"
9,600108,"Broadway S.O, Chennai"


In [4]:
geolocator = Nominatim(user_agent="ny_explorer")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=0.0, max_retries=2, error_wait_seconds=1.0, swallow_exceptions=True, return_value_on_exception=None)
Chennai_data['Location']=Chennai_data['Area'].apply(geocode)
Chennai_data.head(10)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Kasturibai Nagar S.O, Chennai',), **{}).
Traceback (most recent call last):
  File "C:\Users\Shashi\AppData\Local\Continuum\anaconda3\lib\site-packages\geopy\geocoders\base.py", line 355, in _call_geocoder
    page = requester(req, timeout=timeout, **kwargs)
  File "C:\Users\Shashi\AppData\Local\Continuum\anaconda3\lib\urllib\request.py", line 525, in open
    response = self._open(req, data)
  File "C:\Users\Shashi\AppData\Local\Continuum\anaconda3\lib\urllib\request.py", line 543, in _open
    '_open', req)
  File "C:\Users\Shashi\AppData\Local\Continuum\anaconda3\lib\urllib\request.py", line 503, in _call_chain
    result = func(*args)
  File "C:\Users\Shashi\AppData\Local\Continuum\anaconda3\lib\urllib\request.py", line 1360, in https_open
    context=self._context, check_hostname=self._check_hostname)
  File "C:\Users\Shashi\AppData\Local\Continuum\anaconda3\lib\urllib\request.py", line 1320, in do_open
    r = h.g

,Pincode,Area,Location
0,600029,"Aminjikarai S.O, Chennai","(Aminjikarai, Zone 8 Anna Nagar, சென்னை - Chen..."
1,600102,"Anna Nagar East S.O, Chennai",None
2,600040,"Anna Nagar S.O (Chennai), Chennai","(Anna Nagar, Zone 1 Tiruvottiyur, Kathivakkam,..."
3,600101,"Anna Nagar Western Extn S.O, Chennai",None
4,600002,"Anna Road H.O, Chennai","(Anna Road, Sekkadu, Avadi, Poonamallee, Thiru..."
5,600106,"Arumbakkam S.O, Chennai","(Arumbakkam, Uthukkottai, Thiruvallur district..."
6,600083,"Ashoknagar S.O (Chennai), Chennai",None
7,600023,"Aynavaram S.O, Chennai",None
8,600090,"Besantnagar S.O, Chennai",None
9,600108,"Broadway S.O, Chennai",None


In [5]:
#Chennai_data=pd.read_csv('Chennai.txt') #remember to remove it while uploading
Chennai_data['Latitude'] = Chennai_data['Location'].apply(lambda loc: tuple(loc.point)[0] if loc else None)
Chennai_data['Longitude'] = Chennai_data['Location'].apply(lambda loc: tuple(loc.point)[1] if loc else None)
Chennai_data.dropna(inplace= True)
print(Chennai_data.shape)
#print(Chennai_data1.shape)
Chennai_data.head(10)

(67, 5)


,Pincode,Area,Location,Latitude,Longitude
0,600029,"Aminjikarai S.O, Chennai","(Aminjikarai, Zone 8 Anna Nagar, சென்னை - Chen...",13.072140,80.220545
2,600040,"Anna Nagar S.O (Chennai), Chennai","(Anna Nagar, Zone 1 Tiruvottiyur, Kathivakkam,...",13.207371,80.315029
4,600002,"Anna Road H.O, Chennai","(Anna Road, Sekkadu, Avadi, Poonamallee, Thiru...",13.120797,80.094831
5,600106,"Arumbakkam S.O, Chennai","(Arumbakkam, Uthukkottai, Thiruvallur district...",13.233998,79.967376
13,600005,"Chepauk S.O, Chennai","(Chepauk, Ward 114, Zone 9 Teynampet, சென்னை -...",13.063630,80.281278
15,600002,"Chintadripet S.O, Chennai","(Chintadripet, Zone 5 Royapuram, சென்னை - Chen...",13.076126,80.270781
16,600112,"Choolai S.O, Chennai","(Choolai, Zone 6 Thiru. Vi. Ka. Nagar, சென்னை ...",13.090628,80.262142
17,600094,"Choolaimedu S.O, Chennai","(Choolaimedu, Zone 9 Teynampet, சென்னை - Chenn...",13.062334,80.225401
20,600003,"Dr. Ambedkar Nagar S.O (Chennai), Chennai","(Dr Ambedkar Nagar, Zone 6 Thiru. Vi. Ka. Naga...",13.125383,80.207649
23,600008,"Egmore S.O, Chennai","(Egmore, Zone 5 Royapuram, சென்னை - Chennai, C...",13.072832,80.257691


Lets Explore the Regions in Chennai, India

In [6]:
address = 'Chennai Park, Chennai, India'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [7]:
# create map of Chennai using latitude and longitude values
map_Chennai = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(Chennai_data['Latitude'], Chennai_data['Longitude'], Chennai_data['Area']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Chennai)  
    
map_Chennai

#### Defining Foursquare Credentials and Version

In [8]:
CLIENT_ID = 'ILA30HSV5MS1XD04JHRFC05AD21FSJK2R0B2002P2IYQCLX3' # your Foursquare ID
CLIENT_SECRET = 'CHD0IUG3VJNHL0QSASHIESUOJVC0BLKJFOJET3VS2ZTHKBGQ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT= 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ILA30HSV5MS1XD04JHRFC05AD21FSJK2R0B2002P2IYQCLX3
CLIENT_SECRET:CHD0IUG3VJNHL0QSASHIESUOJVC0BLKJFOJET3VS2ZTHKBGQ


In [9]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

##### Creating function for getting venues of near Chennai Park, Chennai, India

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Area', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
Chennai_venues = getNearbyVenues(names=Chennai_data['Area'],
                                   latitudes=Chennai_data['Latitude'],
                                   longitudes=Chennai_data['Longitude']
                                  )
print(Chennai_venues.shape)
Chennai_venues.head()

(1883, 7)


,Area,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Aminjikarai S.O, Chennai",13.07214,80.220545,McDonald's,13.073759,80.221388,Fast Food Restaurant
1,"Aminjikarai S.O, Chennai",13.07214,80.220545,Westside,13.073687,80.221380,Clothing Store
2,"Aminjikarai S.O, Chennai",13.07214,80.220545,@home,13.071499,80.222309,Furniture / Home Store
3,"Aminjikarai S.O, Chennai",13.07214,80.220545,PVR,13.073799,80.221392,Multiplex
4,"Aminjikarai S.O, Chennai",13.07214,80.220545,Landmark,13.073781,80.221393,Bookstore


In [20]:
#Chennai_venues.to_csv('Venue_list.csv')
#Chennai_venues=pd.read_csv('Venue_list.csv')
Chennai_venues.groupby('Area').count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Area,,,,,,
"Aminjikarai S.O, Chennai",20,20,20,20,20,20
"Anna Nagar S.O (Chennai), Chennai",2,2,2,2,2,2
"Anna Road H.O, Chennai",6,6,6,6,6,6
"Chepauk S.O, Chennai",15,15,15,15,15,15
"Chintadripet S.O, Chennai",23,23,23,23,23,23
"Choolai S.O, Chennai",5,5,5,5,5,5
"Choolaimedu S.O, Chennai",15,15,15,15,15,15
"Dr. Ambedkar Nagar S.O (Chennai), Chennai",6,6,6,6,6,6
"Egmore S.O, Chennai",49,49,49,49,49,49


#### Analysing Each Location

In [21]:
# one hot encoding
Chennai_onehot = pd.get_dummies(Chennai_venues[['Venue Category']], prefix="", prefix_sep="")

# add location column back to dataframe
Chennai_onehot['Area'] = Chennai_venues['Area'] 

# move location column to the first column
fixed_columns = [Chennai_onehot.columns[-1]] + list(Chennai_onehot.columns[:-1])
Chennai_onehot = Chennai_onehot[fixed_columns]

Chennai_grouped =Chennai_onehot.groupby('Area').mean().reset_index()
Chennai_grouped.head()

,Area,ATM,African Restaurant,Airport,American Restaurant,Amphitheater,Antique Shop,Arcade,Arts & Crafts Store,Asian Restaurant,...,Thai Restaurant,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Video Store,Watch Shop,Whisky Bar,Women's Store,Yoga Studio
0,"Aminjikarai S.O, Chennai",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0
1,"Anna Nagar S.O (Chennai), Chennai",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.500000,0.000000,0.0,0.0,0.0,0.0,0.0
2,"Anna Road H.O, Chennai",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.166667,0.000000,0.0,0.0,0.0,0.0,0.0
3,"Chepauk S.O, Chennai",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.066667,0.066667,0.0,0.0,0.0,0.0,0.0
4,"Chintadripet S.O, Chennai",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.086957,0.086957,0.0,0.0,0.0,0.0,0.0


In [22]:
#Chennai_venues['Venue Category']
#print(Chennai_grouped.filter(regex='Restaurant$',axis=1))


#### Let's print each location along with the top 5 most common venues

In [23]:
num_top_venues = 5

for area in Chennai_grouped['Area']:
    print("----"+area+"----")
    temp = Chennai_grouped[Chennai_grouped['Area'] == area].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aminjikarai S.O, Chennai----
                  venue  freq
0  Fast Food Restaurant  0.15
1           Pizza Place  0.10
2            Playground  0.05
3             Bookstore  0.05
4           Event Space  0.05


----Anna Nagar S.O (Chennai), Chennai----
                        venue  freq
0          Burmese Restaurant   0.5
1               Train Station   0.5
2                         ATM   0.0
3                 Music Store   0.0
4  Modern European Restaurant   0.0


----Anna Road H.O, Chennai----
               venue  freq
0  Indian Restaurant  0.33
1   Department Store  0.33
2      Movie Theater  0.17
3      Train Station  0.17
4        Music Store  0.00


----Chepauk S.O, Chennai----
                       venue  freq
0          Indian Restaurant  0.27
1              Memorial Site  0.13
2                      Beach  0.07
3         Light Rail Station  0.07
4  Middle Eastern Restaurant  0.07


----Chintadripet S.O, Chennai----
                           venue  freq
0               

                           venue  freq
0              Indian Restaurant  0.19
1                          Hotel  0.12
2                      Bookstore  0.08
3                       Platform  0.08
4  Vegetarian / Vegan Restaurant  0.08


----Perambur High Road S.O, Chennai----
                           venue  freq
0                      Multiplex  0.29
1                  Shopping Mall  0.14
2                    Pizza Place  0.14
3  Vegetarian / Vegan Restaurant  0.14
4             Chinese Restaurant  0.14


----Perambur North S.O, Chennai----
                           venue  freq
0                      Multiplex  0.25
1                    Snack Place  0.12
2  Vegetarian / Vegan Restaurant  0.12
3             Chinese Restaurant  0.12
4               Asian Restaurant  0.12


----Perambur S.O, Chennai----
                           venue  freq
0                      Multiplex  0.25
1                    Snack Place  0.12
2  Vegetarian / Vegan Restaurant  0.12
3             Chinese Restaura

In [24]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Area']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Area_venues_sorted = pd.DataFrame(columns=columns)
Area_venues_sorted['Area'] = Chennai_grouped['Area']

for ind in np.arange(Chennai_grouped.shape[0]):
    Area_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Chennai_grouped.iloc[ind, :], num_top_venues)

Area_venues_sorted.head()

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Aminjikarai S.O, Chennai",Fast Food Restaurant,Pizza Place,Event Space,Furniture / Home Store,Clothing Store,Men's Store,Bakery,Playground,Burger Joint,Electronics Store
1,"Anna Nagar S.O (Chennai), Chennai",Burmese Restaurant,Train Station,Yoga Studio,Farmers Market,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Field
2,"Anna Road H.O, Chennai",Indian Restaurant,Department Store,Train Station,Movie Theater,Farmers Market,Food & Drink Shop,Food,Flower Shop,Flea Market,Field
3,"Chepauk S.O, Chennai",Indian Restaurant,Memorial Site,Beach,Music Store,Platform,Middle Eastern Restaurant,Light Rail Station,Seafood Restaurant,Cricket Ground,Vegetarian / Vegan Restaurant
4,"Chintadripet S.O, Chennai",Hotel,Indian Restaurant,Movie Theater,Bookstore,Vegetarian / Vegan Restaurant,Train Station,Sandwich Place,Flea Market,Fast Food Restaurant,Farmers Market


#### Clustering Areas

In [26]:
# set number of clusters
kclusters = 5

Chennai_grouped_clustering = Chennai_grouped.drop('Area', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Chennai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 2, 1, 1, 1, 1, 0, 1, 1, 0])

In [27]:
# add clustering labels
Area_venues_sorted.insert(0, 'Labels', kmeans.labels_)

Chennai_merged = Chennai_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Chennai_merged = Chennai_merged.join(Area_venues_sorted.set_index('Area'), on='Area')
Chennai_merged.dropna(inplace= True)
Chennai_merged['Labels']=Chennai_merged['Labels'].astype(int)
Chennai_merged.head() # check the last columns!

,Pincode,Area,Location,Latitude,Longitude,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,600029,"Aminjikarai S.O, Chennai","(Aminjikarai, Zone 8 Anna Nagar, சென்னை - Chen...",13.072140,80.220545,0,Fast Food Restaurant,Pizza Place,Event Space,Furniture / Home Store,Clothing Store,Men's Store,Bakery,Playground,Burger Joint,Electronics Store
2,600040,"Anna Nagar S.O (Chennai), Chennai","(Anna Nagar, Zone 1 Tiruvottiyur, Kathivakkam,...",13.207371,80.315029,2,Burmese Restaurant,Train Station,Yoga Studio,Farmers Market,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Field
4,600002,"Anna Road H.O, Chennai","(Anna Road, Sekkadu, Avadi, Poonamallee, Thiru...",13.120797,80.094831,1,Indian Restaurant,Department Store,Train Station,Movie Theater,Farmers Market,Food & Drink Shop,Food,Flower Shop,Flea Market,Field
13,600005,"Chepauk S.O, Chennai","(Chepauk, Ward 114, Zone 9 Teynampet, சென்னை -...",13.063630,80.281278,1,Indian Restaurant,Memorial Site,Beach,Music Store,Platform,Middle Eastern Restaurant,Light Rail Station,Seafood Restaurant,Cricket Ground,Vegetarian / Vegan Restaurant
15,600002,"Chintadripet S.O, Chennai","(Chintadripet, Zone 5 Royapuram, சென்னை - Chen...",13.076126,80.270781,1,Hotel,Indian Restaurant,Movie Theater,Bookstore,Vegetarian / Vegan Restaurant,Train Station,Sandwich Place,Flea Market,Fast Food Restaurant,Farmers Market


### Now let's visualize the Location clusters

In [28]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Chennai_merged['Latitude'], Chennai_merged['Longitude'], Chennai_merged['Area'], Chennai_merged['Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examining each clusters

Cluster 1

In [29]:
Chennai_merged.loc[Chennai_merged['Labels'] == 0, Chennai_merged.columns[[1] + list(range(5, Chennai_merged.shape[1]))]]

,Area,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Aminjikarai S.O, Chennai",0,Fast Food Restaurant,Pizza Place,Event Space,Furniture / Home Store,Clothing Store,Men's Store,Bakery,Playground,Burger Joint,Electronics Store
17,"Choolaimedu S.O, Chennai",0,Indian Restaurant,Department Store,Gym,Ice Cream Shop,Train Station,Restaurant,Pizza Place,Mobile Phone Shop,Chinese Restaurant,Burger Joint
27,"Ethiraj Salai S.O, Chennai",0,Indian Restaurant,Café,Middle Eastern Restaurant,Multiplex,Clothing Store,Hotel,Fast Food Restaurant,Juice Bar,Movie Theater,Coffee Shop
28,"Flower Bazaar S.O, Chennai",0,Indian Restaurant,Restaurant,Market,Fast Food Restaurant,Convenience Store,Video Store,Harbor / Marina,Train Station,Department Store,Snack Place
30,"Fort St George S.O, Chennai",0,Indian Restaurant,Bus Station,Historic Site,Convenience Store,Train Station,Harbor / Marina,Furniture / Home Store,Campground,Museum,Farmers Market
33,"Gopalapuram S.O (Chennai), Chennai",0,Indian Restaurant,Café,Multiplex,Hotel,Ice Cream Shop,Juice Bar,Chinese Restaurant,Concert Hall,Movie Theater,Middle Eastern Restaurant
35,"Greams Road S.O, Chennai",0,Indian Restaurant,Café,Hotel,Multiplex,Middle Eastern Restaurant,Bakery,Fast Food Restaurant,Coffee Shop,Pizza Place,Chinese Restaurant
37,"Guindy North S.O, Chennai",0,Indian Restaurant,Hotel,Restaurant,Spa,Dessert Shop,Diner,Molecular Gastronomy Restaurant,Pizza Place,Coffee Shop,Pool Hall
39,"Hindi Prachar Sabha S.O, Chennai",0,Indian Restaurant,Hotel,Clothing Store,Restaurant,Italian Restaurant,Lounge,Pizza Place,Café,Shopping Mall,Park
40,"Icf Colony S.O, Chennai",0,Indian Restaurant,Fast Food Restaurant,Department Store,Café,Juice Bar,Bakery,Bistro,Ice Cream Shop,Gym / Fitness Center,Market


Cluster 2

In [30]:
Chennai_merged.loc[Chennai_merged['Labels'] == 1, Chennai_merged.columns[[1] + list(range(5, Chennai_merged.shape[1]))]]

,Area,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,"Anna Road H.O, Chennai",1,Indian Restaurant,Department Store,Train Station,Movie Theater,Farmers Market,Food & Drink Shop,Food,Flower Shop,Flea Market,Field
13,"Chepauk S.O, Chennai",1,Indian Restaurant,Memorial Site,Beach,Music Store,Platform,Middle Eastern Restaurant,Light Rail Station,Seafood Restaurant,Cricket Ground,Vegetarian / Vegan Restaurant
15,"Chintadripet S.O, Chennai",1,Hotel,Indian Restaurant,Movie Theater,Bookstore,Vegetarian / Vegan Restaurant,Train Station,Sandwich Place,Flea Market,Fast Food Restaurant,Farmers Market
16,"Choolai S.O, Chennai",1,Indian Restaurant,Hotel,Pizza Place,Department Store,Juice Bar,Event Space,Food & Drink Shop,Food,Flower Shop,Flea Market
20,"Dr. Ambedkar Nagar S.O (Chennai), Chennai",1,Indian Restaurant,Intersection,Department Store,Electronics Store,Hotel,ATM,Train Station,Watch Shop,Food,Flower Shop
23,"Egmore S.O, Chennai",1,Indian Restaurant,Hotel,Café,Seafood Restaurant,Pizza Place,Farmers Market,Donut Shop,Middle Eastern Restaurant,Mexican Restaurant,Coffee Shop
51,"Kosapet S.O (Chennai), Chennai",1,Indian Restaurant,Hotel,Vegetarian / Vegan Restaurant,Department Store,Farmers Market,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market
75,"Park Town H.O, Chennai",1,Indian Restaurant,Hotel,Vegetarian / Vegan Restaurant,Train Station,Platform,Bookstore,Market,Furniture / Home Store,Outlet Store,Electronics Store
81,"Pudupakkam S.O, Chennai",1,Indian Restaurant,Vegetarian / Vegan Restaurant,Restaurant,Coffee Shop,Asian Restaurant,Farmers Market,Food & Drink Shop,Food,Flower Shop,Flea Market
82,"Pudupet S.O (Chennai), Chennai",1,Indian Restaurant,Hotel,Café,Multiplex,Bowling Alley,Movie Theater,Dessert Shop,Farmers Market,General Entertainment,Bookstore


Cluster 3

In [31]:
Chennai_merged.loc[Chennai_merged['Labels'] == 2, Chennai_merged.columns[[1] + list(range(5, Chennai_merged.shape[1]))]]

,Area,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Anna Nagar S.O (Chennai), Chennai",2,Burmese Restaurant,Train Station,Yoga Studio,Farmers Market,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Field
34,"Govt Stanley Hospital S.O, Chennai",2,Train Station,Indian Restaurant,Department Store,Ice Cream Shop,Hookah Bar,Event Space,Food & Drink Shop,Food,Flower Shop,Flea Market
123,"Washermanpet S.O, Chennai",2,Train Station,Women's Store,Vegetarian / Vegan Restaurant,Department Store,Yoga Studio,Farmers Market,Food Court,Food & Drink Shop,Food,Flower Shop


Cluster 4

In [32]:
Chennai_merged.loc[Chennai_merged['Labels'] == 3, Chennai_merged.columns[[1] + list(range(5, Chennai_merged.shape[1]))]]

,Area,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
109,"Tondiarpet S.O, Chennai",3,Platform,Train Station,Harbor / Marina,Spa,Yoga Studio,Event Space,Food & Drink Shop,Food,Flower Shop,Flea Market
116,"Venkatesapuram S.O, Chennai",3,Platform,Costume Shop,Pizza Place,Train Station,Light Rail Station,Electronics Store,Food,Flower Shop,Flea Market,Field
121,"Vyasarpadi S.O, Chennai",3,ATM,Platform,Train Station,Bakery,Event Space,Food & Drink Shop,Food,Flower Shop,Flea Market,Field


Cluster 5

In [33]:
Chennai_merged.loc[Chennai_merged['Labels'] == 4, Chennai_merged.columns[[1] + list(range(5, Chennai_merged.shape[1]))]]

,Area,Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,"Koyambedu S.O, Chennai",4,Fast Food Restaurant,Shopping Mall,Multiplex,Vegetarian / Vegan Restaurant,Food Court,Department Store,Coffee Shop,Clothing Store,Movie Theater,Flower Shop
77,"Perambur High Road S.O, Chennai",4,Multiplex,Vegetarian / Vegan Restaurant,Chinese Restaurant,Asian Restaurant,Pizza Place,Shopping Mall,Department Store,Daycare,Food & Drink Shop,Food
78,"Perambur North S.O, Chennai",4,Multiplex,Shopping Mall,Snack Place,Vegetarian / Vegan Restaurant,Pizza Place,Asian Restaurant,Chinese Restaurant,Farmers Market,Food & Drink Shop,Food
79,"Perambur S.O, Chennai",4,Multiplex,Shopping Mall,Snack Place,Vegetarian / Vegan Restaurant,Pizza Place,Asian Restaurant,Chinese Restaurant,Farmers Market,Food & Drink Shop,Food


##### From the above analysis, I will go forward with two clusters. The above decision goes to the restaurant owner as if he wishes to open an Indian themed restaurant than He can go with cluster with highest number of indian restaurants. If he decides to open other themed restaurants and wishes his restaurant is near to all Public facilies as well, then He can go with the other cluster having restaurants in it.